Необходимо проанализировать данные, полученные с обучающей платформы, которые включают в себя информацию об оценках в тесте, список предметов по семестрам, результаты тестов студентов, информацию о времени регистрации. Дополнительно построить адаптированные RFM-кластеры студентов, чтобы качественно оценить аудиторию.


In [88]:
import pandas as pd
assessments=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-j-semenova/Project_1/assessments.csv')
courses=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-j-semenova/Project_1/courses.csv')
studentAssessment=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-j-semenova/Project_1/studentAssessment.csv')
studentRegistration=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-j-semenova/Project_1/studentRegistration.csv')

1. Необходимо выявить, сколько студентов успешно сдали только один курс, под успешной слачей подразумевается зачёт по курсу на экзамене.

In [89]:
assessments_studentAssessment=pd.merge(assessments, studentAssessment, on='id_assessment')

In [90]:
assessments_studentAssessment.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79.0


Вывела тесты, которые являются экзаменами+только те, которые сданы, т.е score>=40

In [91]:
Exam_passed=assessments_studentAssessment.query('assessment_type=="Exam" and score>=40')

In [92]:
Exam_passed.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0


Количество сданных экзаменов по каждому студенту:

In [93]:
Exam_passed_student=Exam_passed.groupby('id_student', as_index=False).agg({'assessment_type':'count'}).sort_values('assessment_type')

In [94]:
Exam_passed_student

,id_student,assessment_type
0,23698,1
2694,627399,1
2695,627401,1
2696,627417,1
2697,627453,1
...,...,...
3612,1442991,2
3610,1440467,2
462,355935,2
1580,563016,2


In [95]:
Exam_passed_student.query('assessment_type==1').shape #студенты сдавшие только один курс

(3802, 2)

ВЫВОД: количество студентов, сдавших только один курс = 3802

2.  Необходимо определить самый сложный и самый простой экзамен, найти курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью

In [96]:
assessments_studentAssessment.code_module.unique()#вывела все предметы

array(['AAA', 'BBB', 'CCC', 'DDD', 'EEE', 'FFF', 'GGG'], dtype=object)

Количество попыток сдать экзамен для каждого предмета, где есть экзамен:

In [97]:
attempt_exam_passed=assessments_studentAssessment.query('assessment_type=="Exam"').groupby('code_module', as_index=False).agg({'score':'count'})

In [98]:
attempt_exam_passed

,code_module,score
0,CCC,1915
1,DDD,3044


Количество сданных экзаменов для каждого предмета (score>=40)

In [99]:
Exam_passed_count=assessments_studentAssessment.query('assessment_type=="Exam" and score>=40').groupby('code_module', as_index=False).agg({'score':'count'})

In [100]:
Exam_passed_count

,code_module,score
0,CCC,1683
1,DDD,2709


Завершаемость CCC

In [101]:
CCC=(Exam_passed_count.score[0]/attempt_exam_passed.score[0]).round(3)

In [102]:
CCC

0.879

Завершаемость DDD

In [103]:
DDD=(Exam_passed_count.score[1]/attempt_exam_passed.score[1]).round(3)

In [104]:
DDD

0.89

ВЫВОД: завершаемость у CCC меньше,чем у DDD

3. По каждому предмету определить средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [105]:
Exam_passed.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0


Cредний срок сдачи экзаменов

In [106]:
date_submitted_mean=Exam_passed.groupby('code_module', as_index=False).agg({'date_submitted':'mean'}).round()

In [107]:
date_submitted_mean

,code_module,date_submitted
0,CCC,239.0
1,DDD,238.0


ВЫВОД: средний срок сдачи экзаменов CCC=239, DDD=238

4. Выявиnm самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3)


In [108]:
studentRegistration.head(5)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


Cамые популярные предметы (ТОП-3) по количеству регистраций на них

In [109]:
studentRegistration_top=studentRegistration.groupby('code_module', as_index=False).agg({'date_registration':'count'}).sort_values('date_registration', ascending=False).head(3)

In [110]:
studentRegistration_top

,code_module,date_registration
1,BBB,7900
5,FFF,7751
3,DDD,6257


Предметы с самым большим оттоком (ТОП-3)

In [111]:
date_unregistration_top=studentRegistration.groupby('code_module', as_index=False).agg({'date_unregistration':'count'}).sort_values('date_unregistration', ascending=False).head(3)


In [112]:
date_unregistration_top

,code_module,date_unregistration
5,FFF,2380
1,BBB,2377
3,DDD,2235


ВЫВОД: ТОП-3 по популярности BBB, FFF, DDD; 
ТОП-3 по оттоку FFF, BBB, DDD

5. В период с начала 2013 по конец 2014 выявить семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

Количество попыток сдать экзамен по семестрам для каждого предмета, где есть экзамен

In [113]:
attempt_exam_passed_semester=assessments_studentAssessment.query('assessment_type=="Exam"').groupby(['code_module','code_presentation'], as_index=False).agg({'score':'count'}).reset_index()

In [114]:
attempt_exam_passed_semester= attempt_exam_passed_semester.rename(columns={'score': 'attempt_exam_passed_semester_count_score'})

In [115]:
attempt_exam_passed_semester

,index,code_module,code_presentation,attempt_exam_passed_semester_count_score
0,0,CCC,2014B,747
1,1,CCC,2014J,1168
2,2,DDD,2013B,602
3,3,DDD,2013J,968
4,4,DDD,2014B,524
5,5,DDD,2014J,950


Количество сданных экзаменов для каждого предмета(score>=40) по семестрам

In [116]:
Exam_passed_semester=assessments_studentAssessment.query('assessment_type=="Exam" and score>=40').groupby(['code_module','code_presentation'], as_index=False).agg({'score':'count'}).reset_index()

In [117]:
Exam_passed_semester=Exam_passed_semester.rename(columns={'score': 'Exam_passed_semester_count_score'})

In [118]:
Exam_passed_semester

,index,code_module,code_presentation,Exam_passed_semester_count_score
0,0,CCC,2014B,664
1,1,CCC,2014J,1019
2,2,DDD,2013B,504
3,3,DDD,2013J,878
4,4,DDD,2014B,485
5,5,DDD,2014J,842


In [119]:
merge_for_completeness=pd.merge(Exam_passed_semester, attempt_exam_passed_semester, on='index')

In [120]:
merge_for_completeness

,index,code_module_x,code_presentation_x,Exam_passed_semester_count_score,code_module_y,code_presentation_y,attempt_exam_passed_semester_count_score
0,0,CCC,2014B,664,CCC,2014B,747
1,1,CCC,2014J,1019,CCC,2014J,1168
2,2,DDD,2013B,504,DDD,2013B,602
3,3,DDD,2013J,878,DDD,2013J,968
4,4,DDD,2014B,485,DDD,2014B,524
5,5,DDD,2014J,842,DDD,2014J,950


In [121]:
merge_for_completeness['completeness']=merge_for_completeness.Exam_passed_semester_count_score/merge_for_completeness.attempt_exam_passed_semester_count_score

In [122]:
merge_for_completeness

,index,code_module_x,code_presentation_x,Exam_passed_semester_count_score,code_module_y,code_presentation_y,attempt_exam_passed_semester_count_score,completeness
0,0,CCC,2014B,664,CCC,2014B,747,0.888889
1,1,CCC,2014J,1019,CCC,2014J,1168,0.872432
2,2,DDD,2013B,504,DDD,2013B,602,0.837209
3,3,DDD,2013J,878,DDD,2013J,968,0.907025
4,4,DDD,2014B,485,DDD,2014B,524,0.925573
5,5,DDD,2014J,842,DDD,2014J,950,0.886316


In [123]:
merge_for_completeness.sort_values('completeness')

,index,code_module_x,code_presentation_x,Exam_passed_semester_count_score,code_module_y,code_presentation_y,attempt_exam_passed_semester_count_score,completeness
2,2,DDD,2013B,504,DDD,2013B,602,0.837209
1,1,CCC,2014J,1019,CCC,2014J,1168,0.872432
5,5,DDD,2014J,842,DDD,2014J,950,0.886316
0,0,CCC,2014B,664,CCC,2014B,747,0.888889
3,3,DDD,2013J,878,DDD,2013J,968,0.907025
4,4,DDD,2014B,485,DDD,2014B,524,0.925573


In [124]:
Exam_passed_semester_mean=assessments_studentAssessment.query('assessment_type=="Exam" and score>=40')\
.groupby(['code_module', 'code_presentation'], as_index=False).agg({'date_submitted':'mean'}).sort_values('date_submitted', ascending=False)

In [125]:
Exam_passed_semester_mean

,code_module,code_presentation,date_submitted
1,CCC,2014J,244.404318
5,DDD,2014J,242.804038
3,DDD,2013J,239.509112
4,DDD,2014B,234.936082
0,CCC,2014B,231.581325
2,DDD,2013B,230.164683


ВЫВОД: Самая низкая завершаемость семестр 2013B  для курса DDD; самый долгий средний срок сдачи семестр 2014J для курса ССС

6. Для того, чтобы качественно оценить аудиторю, построить адаптированные RFM-кластеры студентов. В адаптированной кластеризации  использовать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Для каждого RFM-сегмента построить границы метрик recency, frequency и monetary для интерпретации этих кластеров. 

6.1 R - среднее время сдачи одного экзамена. Для подготовки данных использую 2 таблицы, созданные ранее:

In [126]:
Exam_passed.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0


In [127]:
Exam_passed=Exam_passed.drop(columns = ['id_assessment','date','weight','is_banked'],axis = 1)

In [128]:
Exam_passed.head(5)

,code_module,code_presentation,assessment_type,id_student,date_submitted,score
52924,CCC,2014B,Exam,559706,234,78.0
52925,CCC,2014B,Exam,559770,230,54.0
52926,CCC,2014B,Exam,560114,230,64.0
52927,CCC,2014B,Exam,560311,234,100.0
52928,CCC,2014B,Exam,560494,230,92.0


In [129]:
date_submitted_mean#среднее время сдачи экзаменов

,code_module,date_submitted
0,CCC,239.0
1,DDD,238.0


Для разделения студентов на кластеры по времени сдачи экзаменов решила использовать более широкие диапазоны для определения принадлежности к определенной группе. quantile([.3, .7]) - это рамки для средних значений. Всего для R создам 3 группы:

In [130]:
quantile_DDD=Exam_passed.query('code_module=="DDD"')['date_submitted'].quantile([.3, .7])

In [131]:
quantile_DDD

0.3    236.0
0.7    242.0
Name: date_submitted, dtype: float64

In [132]:
quantile_CCC=Exam_passed.query('code_module=="CCC"')['date_submitted'].quantile([.3, .7])

In [133]:
quantile_CCC

0.3    234.0
0.7    244.0
Name: date_submitted, dtype: float64

Считаю, что средние сроки для двух экзаменов отличаются незначительно,поэтому:

In [134]:
quantile=Exam_passed['date_submitted'].quantile([.3, .7])

In [135]:
quantile

0.3    234.0
0.7    243.0
Name: date_submitted, dtype: float64

Определила Recency с разделением на 3 группы (от минимального времени (лучшего) '3' к худшему '1')

In [136]:
Exam_passed['Recency']=Exam_passed.date_submitted.apply(lambda x:'1' if x > 243 else '3' if x < 234 else '2')

In [137]:
Exam_passed.head(5)

,code_module,code_presentation,assessment_type,id_student,date_submitted,score,Recency
52924,CCC,2014B,Exam,559706,234,78.0,2
52925,CCC,2014B,Exam,559770,230,54.0,3
52926,CCC,2014B,Exam,560114,230,64.0,3
52927,CCC,2014B,Exam,560311,234,100.0,2
52928,CCC,2014B,Exam,560494,230,92.0,3


6.2 F - завершаемость курсов. Так как выше я работала только со студентами, которые завершили экзамен (т.е. завершаемость 100%), то данные параметр не беру для дальнейшего анализа (по умолчанию у всех студетов = '3')

6.3 M - среднее количество баллов, получаемое за экзамен. Так же делю студентов на 3 группы по количеству баллов за экзамен:

In [138]:
quantile1=Exam_passed['score'].quantile([.3, .7])

In [139]:
quantile1

0.3    58.0
0.7    82.0
Name: score, dtype: float64

Определила Monetary с разделением на 3 группы (от максимального балла (лучшего) '3' к худшему '1')

In [140]:
Exam_passed['Monetary']=Exam_passed.score.apply(lambda x:'1' if x < 58 else '3' if x > 82 else '2')

In [141]:
Exam_passed.head(5)

,code_module,code_presentation,assessment_type,id_student,date_submitted,score,Recency,Monetary
52924,CCC,2014B,Exam,559706,234,78.0,2,2
52925,CCC,2014B,Exam,559770,230,54.0,3,1
52926,CCC,2014B,Exam,560114,230,64.0,3,2
52927,CCC,2014B,Exam,560311,234,100.0,2,3
52928,CCC,2014B,Exam,560494,230,92.0,3,3


Разделила всех студентов на 3 сегмента: at risk, champions, average. Посчитала, что количество набранных баллов ценится выше, чем срок сдачи экзамена, поэтому определила '13' в champions, а '31' в average

In [142]:
segt_map = {r'11': 'at risk',
    r'13': 'champions',
    r'31': 'average',
    r'[1-2][1-2]': 'average',
    r'[2-3][2-3]': 'champions'}
Exam_passed['Segment'] = Exam_passed['Recency'].map(str) + Exam_passed['Monetary']
Exam_passed['Segment'] = Exam_passed['Segment'].replace(segt_map, regex=True)

In [143]:
Exam_passed.head(5)

,code_module,code_presentation,assessment_type,id_student,date_submitted,score,Recency,Monetary,Segment
52924,CCC,2014B,Exam,559706,234,78.0,2,2,average
52925,CCC,2014B,Exam,559770,230,54.0,3,1,average
52926,CCC,2014B,Exam,560114,230,64.0,3,2,champions
52927,CCC,2014B,Exam,560311,234,100.0,2,3,champions
52928,CCC,2014B,Exam,560494,230,92.0,3,3,champions


In [144]:
COUNT=Exam_passed.Segment.count()

In [145]:
Exam_passed_Segment=Exam_passed.groupby('Segment').agg({'assessment_type':'count'}).reset_index()

In [146]:
Exam_passed_Segment

,Segment,assessment_type
0,at risk,232
1,average,2591
2,champions,1569


In [147]:
Percent_Segment=(Exam_passed_Segment.assessment_type*100/COUNT).round(2)

In [148]:
Percent_Segment

0     5.28
1    58.99
2    35.72
Name: assessment_type, dtype: float64

ВЫВОД: после оценки аудитории и разбивки на сегменты я получила следующие результаты: сегмент at risk составляет около 5% от общего количества, average - 59%, champions - 36%